# 0. Configuration

In [ ]:
MODEL_NAME = "google/flan-t5-large"

BASE_LOCATION = "/content/drive/My Drive/"
DATA_LOCATION = BASE_LOCATION + "dataset-decomposed.jsonl"

LEARNING_RATE = 1e-5
BATCH_SIZE = 4
NUM_EPOCHS = 10

# 1. Install dependencies

In [ ]:
!pip install -q --upgrade transformers datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 2. Load model

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# 3. Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from datasets import Dataset, load_dataset
from ast import literal_eval

# train_dataset = pd.read_csv(TRAIN_LOCATION, index_col=0)
# train_dataset['evidences'] = train_dataset['evidences'].apply(literal_eval)
# train_dataset['scores'] = train_dataset['scores'].apply(literal_eval)
# train_dataset = Dataset.from_pandas(train_dataset)

dataset = load_dataset('json', data_files=[DATA_LOCATION])
dataset = dataset['train'].train_test_split(test_size=0.1)
train_dataset = dataset['train']
val_dataset = dataset['test']

# train()

# val_dataset = pd.read_csv(VAL_LOCATION, index_col=0)
# val_dataset['evidences'] = val_dataset['evidences'].apply(literal_eval)
# val_dataset['scores'] = val_dataset['scores'].apply(literal_eval)
# val_dataset = Dataset.from_pandas(val_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
val_dataset

Dataset({
    features: ['label', 'claim', 'subquestions', 'subquestions_retrieve', 'claim_evidences', 'subquestions_evidence', 'combined_evidence'],
    num_rows: 100
})

# 4. Preprocess data

In [ ]:
def generate_prompt(claim, evidences, subquestions):
    prompt_begin = (
        f"You are a fact-checker. Your task is to classify the claim based on the following evidence.\n\n"
        f"Claim: {claim}\n\n"
        f"{subquestions}\n\n"
        f"Evidences:\n"
    )
    prompt_evidences = f""
    prompt_end = f"\n\nLabel (TRUE, FALSE, CONFLICTING):"

    for k in range(0, min(5, len(evidences))):
        new_evidence = f"\n{k + 1}. {evidences[k]}"
        tokens = tokenizer(prompt_begin + prompt_evidences + new_evidence + prompt_end, max_length=512, truncation=True, return_tensors="pt")
        if tokens['input_ids'].shape[1] < tokenizer.model_max_length:
            prompt_evidences += new_evidence
        else:
            break
    return prompt_begin + prompt_evidences + prompt_end

def preprocess_function(examples):
    inputs = [generate_prompt(examples['claim'][i], examples['combined_evidence'][i], '\n'.join(examples['subquestions'][i])) for i in range(len(examples["claim"]))]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["label"], max_length=16, padding='max_length')
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_validation = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# train_dataset['combined_evidence'][0][0]

print(generate_prompt(train_dataset['claim'][0], train_dataset['combined_evidence'][0], '\n'.join(train_dataset['subquestions'][0])))

You are a fact-checker. Your task is to classify the claim based on the following evidence.

Claim: A 29-year-old San Diego Zoo intern was arrested and charged with attempting sexually assault a gorilla.

Question 0: Where was the gorilla being held?
Question 1: Where was the San Diego Zoo intern charged with?
Question 2: Was #2 located within the San Diego Zoo?

Evidences:

1. a gorilla in the kano state zoological gardens has been accused of swallowing n6.8 million. according to bbc pidgin, a radio station in kano, freedom radio claimed that one of the finance officers said that a gorilla swallowed the missing money.
2. ... intern in the zoo of san diego. he has been formally accused of repeated sexual assault on a 15 years old male gorilla named big george which is one of the ...
3. dec 13, 2020  san diego, california | a san diego zoo employee has been formally charged with repeated sexual assault on a 15-year-old male gorilla.
4. ... san diego zoo employee has been formally accuse

# 4. Prepare training

In [ ]:
from datasets import load_metric
import numpy as np

metric = load_metric("f1")

def to_number(label):
  match label:
    case "TRUE":
      return 0
    case "FALSE":
      return 1
    case "CONFLICTING":
      return 2
    case _:
      return 3

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [to_number(pred) for pred in decoded_preds]
    decoded_labels = [to_number(label) for label in decoded_labels]

    f1_weighted = round(metric.compute(predictions=decoded_preds, references=decoded_labels, average="weighted")['f1'], 4)
    f1_macro = round(metric.compute(predictions=decoded_preds, references=decoded_labels, average="macro")['f1'], 4)
    return {"f1_weighted": f1_weighted, "f1_macro": f1_macro}

<ipython-input-8-70406d3cef66>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


The repository for f1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/f1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
import torch

# Step 4: Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    weight_decay=0.01,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Step 5: Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# 5. Train

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1 Weighted,F1 Macro
1,No log,0.135747,0.642900,0.285700
2,No log,0.048098,0.642900,0.285700
3,4.021800,0.052360,0.642900,0.285700
4,4.021800,0.047938,0.642900,0.285700
5,0.064900,0.045456,0.642900,0.285700
6,0.064900,0.045087,0.642900,0.285700
7,0.059100,0.043686,0.642900,0.285700


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control

Epoch,Training Loss,Validation Loss,F1 Weighted,F1 Macro
1,No log,0.135747,0.642900,0.285700
2,No log,0.048098,0.642900,0.285700
3,4.021800,0.052360,0.642900,0.285700
4,4.021800,0.047938,0.642900,0.285700
5,0.064900,0.045456,0.642900,0.285700
6,0.064900,0.045087,0.642900,0.285700
7,0.059100,0.043686,0.642900,0.285700
8,0.059100,0.043167,0.642900,0.285700
9,0.060400,0.042671,0.642900,0.285700
10,0.060400,0.042436,0.642900,0.285700


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=2250, training_loss=0.9407780096266005, metrics={'train_runtime': 3028.5788, 'train_samples_per_second': 2.972, 'train_steps_per_second': 0.743, 'total_flos': 2.0742911557632e+16, 'train_loss': 0.9407780096266005, 'epoch': 10.0})

In [ ]:
trainer.save_model("/content/drive/My Drive/t5_large")

# 6. Eval

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('/content/drive/My Drive/big_boi')

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Evaluation results: {'eval_loss': 0.042435817420482635, 'eval_f1_weighted': 0.6429, 'eval_f1_macro': 0.2857, 'eval_runtime': 18.3256, 'eval_samples_per_second': 5.457, 'eval_steps_per_second': 1.364, 'epoch': 10.0}
